In [ ]:
import unidecode
import re
from selenium import webdriver
from selenium.webdriver.support.ui import Select, WebDriverWait
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import pygsheets
import os
from fuzzywuzzy import process


In [ ]:

gc = pygsheets.authorize(client_secret='/Users/acapai/Documents/Git/Espace-test/Scrapping-With-Python/Scrapping_Url_GAFEO/Oauth_gg/code_secret_client_95743482524-gj2mnoav9naiqt454ggvt71r28r4n3dk.apps.googleusercontent.com.json')

# Suivi Eron 2022 (derniere version)
#sh = gc.open_by_key('13VqSH8KjAzB3-mroVhtUJjXgO2Gs31UtpqdehiLMyRs')

# DUPLICATA Suivi Eron 2022 (derniere version)
sh = gc.open_by_key('1Ix4xc_kJPrIBXQL8JmGVz_XNnY4t7AMHdHvmyVlExiA')

wksheet_depart_inf_01_01 = sh[4]
col_name_apprenant = wksheet_depart_inf_01_01.get_col(9)
col_formation_raw = wksheet_depart_inf_01_01.get_col(8)

# print(col_name_apprenant)
print(wksheet_depart_inf_01_01.title)
print(sh.title)

col_apprenant = [string for string in col_name_apprenant if string ]
col_formation = [ string.split("-")[0] for string in col_formation_raw if string]


In [ ]:
col_formation_raw = wksheet_depart_inf_01_01.get_col(8)
col_formation = [ string.split("-")[0] for string in col_formation_raw if string]


In [ ]:
datasheet = pd.DataFrame(list(zip(col_formation[2:],col_apprenant[2:])),columns=['Formation','Apprenant'])
datasheet['Dernier Accès'] = ''
datasheet['Pourcentage']=''
datasheet['Apprenant_Normalize']=''
datasheet["Apprenant_Gafeo"]=""


In [ ]:
#Normalize name 
for index,row in datasheet.iterrows():
    print(unidecode.unidecode(row['Apprenant']))
    print(row['Apprenant'].lower())
    print(unidecode.unidecode(row['Apprenant'].lower()))
    print(re.sub(r'[^a-zA-Z0-9]','',unidecode.unidecode(row['Apprenant'].lower().replace(' ', ''))))
    # row['Apprenant_Normalize']= re.sub(r'[^a-zA-Z0-9]','',unidecode.unidecode(row['Apprenant']).lower().replace(' ', ''))
    row['Apprenant_Normalize']= unidecode.unidecode(row['Apprenant']).lower()

In [ ]:
formation_unique= datasheet.Formation.unique().tolist()


In [ ]:
dic_dataframe={}
for formation in formation_unique:
    dic_dataframe[formation]=datasheet[datasheet["Formation"]==formation]
print(dic_dataframe[formation_unique[-2]])

In [ ]:
formation_key_dic=list(dic_dataframe.keys())
print(formation_key_dic)

In [ ]:
if os.path.exists("/Applications/Internet/Google Chrome.app/Contents/MacOS/Google Chrome"):
        path_google_chrome="/Applications/Internet/Google Chrome.app/Contents/MacOS/Google Chrome"
else:
    path_google_chrome="/Applications/Google Chrome.app/Contents/MacOS/Google Chrome"
options = Options()
options.binary_location = path_google_chrome
options.add_experimental_option("detach", True)
# options.add_argument("--headless")
# options.add_argument("--disable-gpu")
# options.add_experimental_option('prefs', {'download.default_directory':'/Users/acapai/Downloads/'})
# options.add_experimental_option('prefs',{"profile.default_content_setting_values.automatic_downloads": 1})

rel_path="/Applications/chromedriver"
PATH = os.path.abspath(rel_path)
time_out=30
id="automate"
psswd=""
browser = webdriver.Chrome(options=options,executable_path=PATH)
browser.get("https://www.gafeo.fr/my/")
browser.find_element(By.ID,'username').send_keys(id)
browser.find_element(By.ID,'password').send_keys(psswd)
browser.find_element(By.CSS_SELECTOR,"button.btn-primary").click()
div_searchbox = WebDriverWait(browser,10).until(EC.presence_of_element_located((By.ID,"coursesearchbox")))

In [ ]:
time_out=30

In [ ]:
# POUR RETOURNER EN ARRIERE
browser.execute_script("window.history.go(-1)")

In [ ]:
inputs_field = WebDriverWait(browser,10).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR,"#coursesearch .coursesearchbox input")))

In [ ]:
inputs_field[0].send_keys(formation_key_dic[0])
inputs_field[1].click()

In [ ]:
action_link = WebDriverWait(browser,10).until(EC.presence_of_all_elements_located((By.CLASS_NAME,"action-link")))
action_link[0].click()

In [ ]:
div_button_config_course = WebDriverWait(browser,10).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR,".teacherdash a")))
div_button_config_course[0].click()

In [ ]:
WebDriverWait(browser,10).until(EC.presence_of_element_located((By.LINK_TEXT,"Rapports de session"))).click()

In [ ]:
dp_down_list_participant = Select(WebDriverWait(browser,10).until(EC.presence_of_element_located((By.CSS_SELECTOR,"select[name='userid']"))))

In [ ]:
select_box = browser.find_element_by_name("userid") 
options = [x for x in select_box.find_elements(By.TAG_NAME,"option")]

In [ ]:
for i,element in enumerate(dp_down_list_participant.options):
    print(element.get_attribute("value"))
    print(element.get_attribute("text"))
    # element_normalize=re.sub(r'[^a-zA-Z0-9]','',unidecode.unidecode(element.get_attribute("text")).lower().replace(' ', ''))
    element_normalize=re.sub(r'[^a-zA-Z0-9]','',unidecode.unidecode(element.get_attribute("text")).lower())
    list_apprenant= dic_dataframe[formation_key_dic[0]]['Apprenant_Normalize'].tolist()
    print("element_normalize:",element_normalize)
    print("list_apprenant:",list_apprenant)
    Ratios = process.extract(element_normalize,list_apprenant)
    print(Ratios)
    highest = process.extractOne(element_normalize,list_apprenant)
    print(highest)
    if element_normalize in dic_dataframe[formation_key_dic[0]]["Apprenant_Normalize"].tolist():
        print("YES")
    # print("dic_dataframe[formation_key_dic[0]]['Apprenant_Normalize']",dic_dataframe[formation_key_dic[0]]["Apprenant_Normalize"])
    # if element_normalize in dic_dataframe[formation_key_dic[0]]["Apprenant_Normalize"]:
        print("dic_dataframe[formation_key_dic[0]]['Apprenant_Normalize']==element_normalize",dic_dataframe[formation_key_dic[0]]["Apprenant_Normalize"]==element_normalize)
        dic_dataframe[formation_key_dic[0]].loc[dic_dataframe[formation_key_dic[0]]["Apprenant_Normalize"]==element_normalize,"Apprenant_Gafeo"]=element.get_attribute("text")

In [ ]:
pd_copy = dic_dataframe[formation_key_dic[0]]
pd_copy["Ratio_Fuzzy"]=""
pd_copy["Name_Fuzzy"]=""
pd_copy["option_value"]=""
list_apprenant= pd_copy['Apprenant_Normalize'].tolist()
for i,element in enumerate(dp_down_list_participant.options):
    print(element.get_attribute("value"))
    print(element.get_attribute("text"))
    # element_normalize=re.sub(r'[^a-zA-Z0-9]','',unidecode.unidecode(element.get_attribute("text")).lower().replace(' ', ''))
    element_normalize=unidecode.unidecode(element.get_attribute("text")).lower()
    print("element_normalize:",element_normalize)
    print("list_apprenant:",list_apprenant)
    Ratios = process.extract(element_normalize,list_apprenant,limit=len(list_apprenant))
    print(Ratios)
    highest = process.extractOne(element_normalize,list_apprenant)
    print("Highest",highest)
    index= [x for x, y in enumerate(list_apprenant) if y == highest[0]]
    if highest[1]>75:
        print("Ratio",highest[1])
        print("index",index[0])
        pd_copy.iloc[index[0],pd_copy.columns.get_loc("Ratio_Fuzzy")]=highest[1]
        pd_copy.iloc[index[0],pd_copy.columns.get_loc("Name_Fuzzy")]=highest[0]
        pd_copy.iloc[index[0],pd_copy.columns.get_loc("option_value")]=element.get_attribute("value")
        pd_copy.iloc[index[0],pd_copy.columns.get_loc("Apprenant_Gafeo")]=element.get_attribute("text")

    

In [ ]:
for ind in pd_copy.index:
    value = pd_copy["option_value"][ind]
    if pd_copy["Apprenant_Gafeo"][ind] !="":
        lastname=pd_copy["Apprenant_Gafeo"][ind].split()[0]
        firstname=pd_copy["Apprenant_Gafeo"][ind].split()[1]
        print(firstname+" "+lastname)
        print("value option",value)
    if value != "":
        dp_down_list_participant = Select(WebDriverWait(browser,time_out).until(EC.presence_of_element_located((By.CSS_SELECTOR,"select[name='userid']"))))
        dp_down_list_participant.select_by_value(value)
        opt_name = WebDriverWait(browser,time_out).until(EC.presence_of_element_located((By.CSS_SELECTOR,"input[name='go_btn']")))
        opt_name.send_keys(Keys.RETURN)
        WebDriverWait(browser,time_out).until(EC.text_to_be_present_in_element((By.CSS_SELECTOR,".userinfobox h1"),lastname))
        info_pourcentage_list = WebDriverWait(browser,time_out).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR,".mandatory-items div")))
        info_pourcentage_text = info_pourcentage_list[2].text
        pourcentage_only=re.findall(r"^.*?\([^\d]*(\d+)[^\d]*\).*$",info_pourcentage_text)
        print("%:",pourcentage_only[0])

        info_last_access = WebDriverWait(browser,time_out).until(EC.presence_of_element_located((By.CSS_SELECTOR,"#sample-lastcourseaccess .sample-value")))
        print("info_last_access",info_last_access.text)
        pd_copy.loc[ind,'Dernier Accès'] = info_last_access.text
        pd_copy.loc[ind,'Pourcentage']=pourcentage_only[0]
    


In [ ]:
# print(datasheet.loc[pd_copy.index.values,"Dernier Accès"])
# print(datasheet.loc[pd_copy.index.values,"Apprenant"])
datasheet.loc[pd_copy.index.values,"Dernier Accès"] = pd_copy.loc[pd_copy.index.values,"Dernier Accès"]
datasheet.loc[pd_copy.index.values,"Pourcentage"] = pd_copy.loc[pd_copy.index.values,"Pourcentage"]